In [56]:
import numpy as np
from scipy import sparse
from copy import copy
from tqdm.notebook import tqdm

from uniparser_eastern_armenian import EasternArmenianAnalyzer

## Считаем данные и получим характеристики коллекции

### Удалим лишние символы и очистим текст

In [ ]:
def format_text(text):
    all_text=copy(text)
    for spaced in ['.','-',',','!','?','(','—',')','՞','՛','։','՝','՜','’','«','»','*']:
        all_text = all_text.replace(spaced, ' {0} '.format(spaced))

    all_text=all_text.replace('   ',' ')
    all_text=all_text.replace('  ',' ')
    return all_text

In [55]:
lines=[]
with open('david_of_sassoun.txt','r',encoding='utf-8') as f:
    lines.extend(f.readlines())

new_lines=[]
all_text=''
# удалим скобки и корокие предложения
for i,line in enumerate(lines):
    text=line.replace('\n','')
    if len(text)>3 and '[' not in text and ']' not in text:
        new_lines.append(text)

# добавим специальные теги в предложения
for line in new_lines:
    all_text=all_text+' <START> '+line+ ' <END>'

In [57]:
all_text=format_text(all_text)[1:]

corpus_words= [word for word in all_text.split(' ') if word != '']

print(all_text[:500])
print(f'Количество символов {len(all_text)}')
print(f'Количество предложений',len(all_text.split('։')))
print(f'Количество словоформ',len(corpus_words))

<START> Դառնամ , զօղորմին տի տամ * Խանում Ծովինարին , Դառնամ , զօղորմին տի տամ Սանասարին , Բաղդասարին . Դառնամ , զօղորմին տի տամ Քեռի Թորոսիկին . Դառնամ , զօղորմին տի տամ Ականջ արողների ծընողներին ։ <END> <START> ՄԱՍՆ Ա ԿՌԻՎ ԲԱՂԴԱԴԻ ԽԱԼԻՖԱՅԻ ԴԵՄ <END> <START> Ըսկիզբն էր կըռապաշտ Խալիֆան , Մեկ էլ Հայոց Գագիկ թագավոր . Կռապաշտ Խալիֆան Բաղդադ կը նստեր , Գագիկթագավոր ՝ Բերդ - Կապոտին ։ <END> <START> Գագիկ թագավոր ծեր , ալևոր էր . Զինք շա ՛ տ հարըստություն ուներ , Զարմ ու զավակներ չուներ . Մեկ աղջիկ 
Количество символов 374807
Количество предложений 4047
Количество словоформ 89321


### Посчитаем леммы

In [53]:
analyzer = EasternArmenianAnalyzer()

In [58]:
lemmas=[]

for word in tqdm(corpus_words):
    analyses = analyzer.analyze_words(word)[0]
    lemmas.append(analyses.lemma)

print(f'Количество лемм {len(np.unique(lemmas))}')

  0%|          | 0/89321 [00:00<?, ?it/s]

Количество лемм 2164


## Создадим статистическую модель

In [ ]:
k = 1 # adjustable

unique_words = list(np.unique(corpus_words))
# разобьем на сеты длиной k
words_sets = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]

# создадим матрицу из сетов и словоформ
unique_words_sets = list(np.unique(words_sets))
sets_count = len(unique_words_sets)

probab_matrix = np.zeros((sets_count, len(unique_words)))

# словари для поиска индекса слова
unique_word2idx = {word: i for i, word in enumerate(unique_words)}
set2idx = {word: i for i, word in enumerate(unique_words_sets)}


# пройдемся по сетам и заполним матрицу
for i, set in enumerate(tqdm(words_sets[:-k])):

    current_set_idx = set2idx[set]
    next_word_idx = unique_word2idx[corpus_words[i+k]]
    probab_matrix[current_set_idx, next_word_idx] +=1



## Сгенерируем текст

In [ ]:
def generate_sentence(initial_sentence, chain_length=15,alpha=0):
    current_set = initial_sentence.split(' ')

    sentence = copy(initial_sentence)

    for _ in range(chain_length):
        sentence+=' '
        # вектор вероятностей для выбранного сета
        next_word_vector = probab_matrix[set2idx[' '.join(current_set)]] + alpha
        nex_words_probability = next_word_vector/next_word_vector.sum()
        next_word = np.random.choice(unique_words, p=nex_words_probability)

        sentence+=next_word
        current_set = current_set[1:]+[next_word]
    return sentence


In [ ]:
# generate_sentence('<START> Դառնամ')
generate_sentence('մեկ բան')

## Оценим вероятности фраз

In [49]:
def sentence_probability(initial_sentence, alpha=1, V=2176):

    corpus_words= [word for word in initial_sentence.split(' ') if word != '']
    words_sets = [ ' '.join(corpus_words[i:i+k]) for i, _ in enumerate(corpus_words[:-k]) ]


    log_probab=0

    for i,current_set in enumerate(words_sets):

        current_set_idx = set2idx.get(current_set)
        next_word_idx = unique_word2idx.get(corpus_words[i+k])
        # сглаживание Лапласса
        if current_set_idx is not None and next_word_idx is not None:
            word_number = probab_matrix[current_set_idx,next_word_idx]

            next_word_vector = probab_matrix[current_set_idx] + alpha
            nex_words_probability = word_number/next_word_vector.sum()
        else:
            if current_set_idx is not None:
                norm=probab_matrix[current_set_idx].sum()
            else:
                norm=0

            nex_words_probability = 1/(norm+V)

        log_probab+=np.log(nex_words_probability)

    return log_probab


In [ ]:
# k=2
txts=['Բաղդադ կը նստեր ',
      'Ետ քաշվան, կանգնա՛ն։',
      'մի տըվին, ԶԴավիթ առան',
      'Դավիթ չը գնաց',
      'Իր համար թե՛ք']

w_txts=['կը Գագիկ նստեր',
      'Ետ կը, կանգնա՛ն։',
      'տար տըվին, ԶԴավիթ առան',
      'տաշտ չը գնաց',
      'կը համար թե՛ Թալ']

for txt in w_txts:
    log_prob=sentence_probability(txt)
    print('-----')
    print(log_prob)
    print(np.exp(log_prob))

# Перплексия

https://stackoverflow.com/questions/54941966/how-can-i-calculate-perplexity-using-nltk


### Униграмная модель

In [ ]:
pp=0

N=89321
V=2176
for i, set in enumerate(tqdm(probab_matrix)):

    word_number=probab_matrix[i, i]
    # сглажвание лапласса
    if word_number==0:
        word_number+=1
        word_probab=word_number/(probab_matrix[i].sum()+V)
    else:
        word_probab=word_number/probab_matrix[i].sum()

    pp=np.log(word_probab)

l=pp/N
print(f'Perplexity={np.power(np.e, -l)}')

## Биграмная модель

In [ ]:
pp=0

N=89321
V=2176
for i, current_set in enumerate(tqdm(unique_words_sets)):

    w_old, w_current=current_set.split(' ')
    old_idx=unique_word2idx.get(w_old)

    old_word_number=probab_matrix[:, old_idx].sum()
    current_set_number = probab_matrix[set2idx.get(current_set)].sum()

    # сглажвание лапласса
    if current_set_number==0:
        current_set_number+=1
        word_probab=current_set_number/(old_word_number+V)
    elif old_word_number==0:
        word_probab=current_set_number/(old_word_number+V)
    else:
        word_probab=current_set_number/old_word_number

    pp=np.log(word_probab)

l=pp/N
print(f'Perplexity={np.power(np.e, -l)}')